# Network analysis in Senegal

### Objectives
    1)	Use measures of road-based accessibility to identify road segments that, if rehabilitated, would improve agricultural market activities in Senegal, including during flood conditions.
    2)	Gain a better understanding of the accessibility, connectivity, and criticality of roads in Senegal in relationship to agricultural origins, processing & transfer sites, and markets.

To this end, the team will develop an accessibility model which measures the travel time from sites of agricultural production to their nearest populated areas, processing centers, and markets. 

### Datasets for analysis
#### ORIGIN
    1) agriculture: MapSPAM 2017. Measuring value in international dollars.
    2) agriculture: UMD Land Cover 2019 30m. Assign MapSPAM value onto land cover cropland class for more precise origin information.
    3) population: WorldPop 2020, UN-adjusted.
    4) settlement extent: GRID3 2020.
#### DESTINATION
    4) markets: derived from WorldPop 2020 and GRID3 2020 urban clusters.
    5) agricultural processing hubs: to be acquired.
#### TRAVEL ROUTE
    6) roads: OpenStreetMap, July 2021.
    7) elevation: 
#### OBSTACLE
    8) flood: FATHOM. 1-in-10, 20, and 50 year flood return periods. 
#### INTERVENTION
    9) upcoming road projects: AGEROUTE interventions separate from the World Bank-financed project
    10) targeted road projects: critical road segments identified by this accessibility model's baseline outputs


### Model design
#### Basic formula: 
    (a) Off-road driving time from origin to closest road node
    +
    (b) Driving time from road node in (a) to a destination (closeness measured by road segments speeds)

#### Model origin & destination (OD) sets:
    A)	Travel time from an area that has agricultural value/potential to the nearest processing hub (if provided).
    B)	Travel time from an area that has agricultural value/potential to the nearest larger settlement, (“larger” settlement identified using a case-appropriate population metric to be determined).
    C)	Travel time from an area that has agricultural value/potential to the nearest market.
    D)	Travel time from all settlements to the nearest market.
    E)	Travel time from larger settlements to the nearest market.

#### Before/after scenarios for each OD set:
    1)	Pre-project, baseline weather: No inclement weather. Road network status as of November 2021.
    2)	Pre-project, flood: 1-in-10, 1-in-20 and 1-in-50 year flood return period. Road network status as of November 2021.
    3)	Post-project, baseline weather: No inclement weather. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).
    4)	Post-project, flood: 1-in-10 year flood return period. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).

#### Notes:
    --Destinations are expected to be proximal to the road network, so no measure is taken between road and destination.
    --All travel times will be assigned to each model variation’s point of origin; the aggregation up to admin areas is possible if desired.
    --Obstacles & interventions modify the road segment speeds. Basic formula is then applied to the modified road network.


### Prep workspace

In [1]:
import os, sys
GISFolder = os.getcwd()
GISFolder

'C:\\Users\\wb527163\\GEO-Cdrive-Grace'

In [2]:
# Note: needed to reinstall rtree due to geopandas import error. Did so in the console. 
# conda install -c conda-forge rtree=0.9.3

In [3]:
# load and filter osm network (step 1)
import geopandas as gpd
from geopandas import GeoDataFrame
import pandas as pd
import time
sys.path.append(r"C:\Users\wb527163\.conda\envs\geo\GOSTnets-master")
import GOSTnets as gn

In [4]:
import networkx as nx
import osmnx as ox
import numpy as np
import rasterio as rt
import shapely
from shapely.geometry import Point, box
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely import wkt
from shapely.geometry import LineString, MultiLineString, Point
import peartree

In [5]:
#### Might not use these
import fiona
from osgeo import gdal
import importlib
import matplotlib.pyplot as plt
import subprocess, glob

In [6]:
pth = os.path.join(GISFolder, "SEN-Cdrive") # Personal folder system for running model.
pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive'

In [7]:
out_pth = os.path.join(GISFolder, "SEN-Cdrive\outputs") # For storing intermediate outputs from the model.
out_pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive\\outputs'

In [8]:
team_pth = 'R:\\SEN\\GEO' # This is where the unmodified input data is stored. Finalized outputs also housed here.
team_pth

'R:\\SEN\\GEO'

### Prepare and clean the data

#### Return periods from FATHOM: 1-in-10, 20, and 50 year floods.

We are starting with just the 1-in-10 year return period. Joining 1-in-20 and 1-in-50 to the road dataframe was causing file size errors. 20 and 50 will be run in a replicated script.

In [12]:
flood10 = gpd.read_file("SEN-Cdrive/flood/PFU_1in10.shp")
flood10.head(50)

,PFU_1in10,geometry
0,1.0,"POLYGON ((-14.95542 16.73208, -14.95458 16.732..."
1,7.0,"POLYGON ((-14.95458 16.73208, -14.95375 16.732..."
2,6.0,"POLYGON ((-14.95792 16.73125, -14.95708 16.731..."
3,1.0,"POLYGON ((-14.95625 16.73125, -14.95542 16.731..."
4,6.0,"POLYGON ((-14.95542 16.73125, -14.95458 16.731..."
5,4.0,"POLYGON ((-14.95792 16.73042, -14.95708 16.730..."
6,4.0,"POLYGON ((-14.95708 16.73042, -14.95625 16.730..."
7,4.0,"POLYGON ((-14.95625 16.73042, -14.95542 16.730..."
8,3.0,"POLYGON ((-14.95542 16.73042, -14.95458 16.730..."
9,3.0,"POLYGON ((-14.95875 16.72958, -14.95792 16.729..."


In [14]:
gTime = nx.read_gpickle("SEN-Cdrive/gTime.pickle")
gTime_edge = gn.edge_gdf_from_graph(gTime)
gTime_edge

,stnode,endnode,access,ref,name,length,time,junction,osmid,mode,oneway,lanes,tunnel,width,area,highway,maxspeed,bridge,geometry
0,358284990,5217543379,NaN,D 523,D 523,33.127,2.385144,NaN,59618174,drive,False,NaN,NaN,NaN,NaN,unclassified,NaN,NaN,"LINESTRING (-12.32347 12.38119, -12.32368 12.3..."
1,358284990,1888282175,NaN,NaN,NaN,12.832,0.769920,NaN,178482063,drive,False,NaN,NaN,NaN,NaN,tertiary,NaN,NaN,"LINESTRING (-12.32347 12.38119, -12.32351 12.3..."
2,358284990,5329792467,NaN,NaN,NaN,48.781,2.926860,NaN,178482063,drive,False,NaN,NaN,NaN,NaN,tertiary,NaN,NaN,"LINESTRING (-12.32347 12.38119, -12.32317 12.3..."
3,358284993,1888282575,NaN,NaN,NaN,126.577,7.594620,NaN,178470940,drive,False,NaN,NaN,NaN,NaN,tertiary,NaN,NaN,"LINESTRING (-12.28135 12.41380, -12.28205 12.4..."
4,358284993,1888198886,NaN,NaN,NaN,120.578,7.234680,NaN,178470940,drive,False,NaN,NaN,NaN,NaN,tertiary,NaN,NaN,"LINESTRING (-12.28135 12.41380, -12.28061 12.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4012560,9246539941,9246539942,NaN,NaN,NaN,76.742,6.906780,NaN,366052716,drive,False,NaN,NaN,NaN,NaN,residential,NaN,NaN,"LINESTRING (-17.48521 14.72445, -17.48451 14.7..."
4012561,9246539942,9246539941,NaN,NaN,NaN,76.742,6.906780,NaN,366052716,drive,False,NaN,NaN,NaN,NaN,residential,NaN,NaN,"LINESTRING (-17.48451 14.72456, -17.48521 14.7..."
4012562,9246539942,3700438702,NaN,NaN,NaN,0.846,0.076140,NaN,366052716,drive,False,NaN,NaN,NaN,NaN,residential,NaN,NaN,"LINESTRING (-17.48451 14.72456, -17.48451 14.7..."
4012563,9276108905,6048975958,NaN,NaN,Route de l'Aeroport,171.902,8.840674,NaN,177950649,drive,True,2,NaN,NaN,NaN,secondary,NaN,NaN,"LINESTRING (-17.50499 14.74971, -17.50357 14.7..."


In [15]:
gTime_node = gn.node_gdf_from_graph(gTime)

### Update driving times based on flood intersection.

#### Join road network and flood raster into single table.

In [18]:
gTime_edge.reset_index(inplace=True) # To create unique ID and to avoid:  ValueError: cannot reindex from a duplicate axis.
gTime_edge.rename(columns={'index': 'ID_graph'}, inplace=True)
gTime_edge.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4012565 entries, 0 to 4012564
Data columns (total 20 columns):
 #   Column    Dtype   
---  ------    -----   
 0   ID_graph  int64   
 1   stnode    int64   
 2   endnode   int64   
 3   access    object  
 4   ref       object  
 5   name      object  
 6   length    float64 
 7   time      float64 
 8   junction  object  
 9   osmid     int64   
 10  mode      object  
 11  oneway    bool    
 12  lanes     object  
 13  tunnel    object  
 14  width     object  
 15  area      object  
 16  highway   object  
 17  maxspeed  object  
 18  bridge    object  
 19  geometry  geometry
dtypes: bool(1), float64(2), geometry(1), int64(4), object(12)
memory usage: 585.5+ MB


In [19]:
# Spatial join should be on projected GDFs.
gTime_edge = gTime_edge.to_crs("EPSG:31028")
flood10 = flood10.to_crs("EPSG:31028")
gTime_edge.crs == flood10.crs

True

In [20]:
join10 = gpd.sjoin_nearest(gTime_edge, flood10, how="left", max_distance=3) 
join10.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4115534 entries, 0 to 4012564
Data columns (total 22 columns):
 #   Column       Dtype   
---  ------       -----   
 0   ID_graph     int64   
 1   stnode       int64   
 2   endnode      int64   
 3   access       object  
 4   ref          object  
 5   name         object  
 6   length       float64 
 7   time         float64 
 8   junction     object  
 9   osmid        int64   
 10  mode         object  
 11  oneway       bool    
 12  lanes        object  
 13  tunnel       object  
 14  width        object  
 15  area         object  
 16  highway      object  
 17  maxspeed     object  
 18  bridge       object  
 19  geometry     geometry
 20  index_right  float64 
 21  PFU_1in10    float64 
dtypes: bool(1), float64(4), geometry(1), int64(4), object(12)
memory usage: 694.7+ MB


In [21]:
join10

,ID_graph,stnode,endnode,access,ref,name,length,time,junction,osmid,...,lanes,tunnel,width,area,highway,maxspeed,bridge,geometry,index_right,PFU_1in10
0,0,358284990,5217543379,NaN,D 523,D 523,33.127,2.385144,NaN,59618174,...,NaN,NaN,NaN,NaN,unclassified,NaN,NaN,"LINESTRING (790871.607 1370083.257, 790848.290...",NaN,NaN
1,1,358284990,1888282175,NaN,NaN,NaN,12.832,0.769920,NaN,178482063,...,NaN,NaN,NaN,NaN,tertiary,NaN,NaN,"LINESTRING (790871.607 1370083.257, 790867.287...",NaN,NaN
2,2,358284990,5329792467,NaN,NaN,NaN,48.781,2.926860,NaN,178482063,...,NaN,NaN,NaN,NaN,tertiary,NaN,NaN,"LINESTRING (790871.607 1370083.257, 790904.086...",NaN,NaN
3,3,358284993,1888282575,NaN,NaN,NaN,126.577,7.594620,NaN,178470940,...,NaN,NaN,NaN,NaN,tertiary,NaN,NaN,"LINESTRING (795418.079 1373739.218, 795343.307...",NaN,NaN
4,4,358284993,1888198886,NaN,NaN,NaN,120.578,7.234680,NaN,178470940,...,NaN,NaN,NaN,NaN,tertiary,NaN,NaN,"LINESTRING (795418.079 1373739.218, 795497.547...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4012560,4012560,9246539941,9246539942,NaN,NaN,NaN,76.742,6.906780,NaN,366052716,...,NaN,NaN,NaN,NaN,residential,NaN,NaN,"LINESTRING (232231.532 1629246.092, 232307.524...",NaN,NaN
4012561,4012561,9246539942,9246539941,NaN,NaN,NaN,76.742,6.906780,NaN,366052716,...,NaN,NaN,NaN,NaN,residential,NaN,NaN,"LINESTRING (232307.524 1629257.675, 232231.532...",NaN,NaN
4012562,4012562,9246539942,3700438702,NaN,NaN,NaN,0.846,0.076140,NaN,366052716,...,NaN,NaN,NaN,NaN,residential,NaN,NaN,"LINESTRING (232307.524 1629257.675, 232307.393...",NaN,NaN
4012563,4012563,9276108905,6048975958,NaN,NaN,Route de l'Aeroport,171.902,8.840674,NaN,177950649,...,2,NaN,NaN,NaN,secondary,NaN,NaN,"LINESTRING (230131.706 1632066.089, 230285.874...",NaN,NaN


In [22]:
# How many nodes experienced flooding?
pc_flooded = join10["PFU_1in10"].count() / len(join10) * 100

print("No flood crossing at node:", join10["PFU_1in10"].isnull().sum(), "locations", end="\n")
print("Flood crossing at node:", join10["PFU_1in10"].count(), "locations", end="\n")
print("\nPercent flooded:", pc_flooded, "percent", "out of", len(join10), "possible locations")

No flood crossing at node: 3849261 locations
Flood crossing at node: 266273 locations

Percent flooded: 6.469950193583627 percent out of 4115534 possible locations


In [23]:
join10 = join10[['ID_graph', 'stnode', 'endnode', 'time', 'length', 'highway', 'osmid', 'geometry', 'PFU_1in10']]
join10.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4115534 entries, 0 to 4012564
Data columns (total 9 columns):
 #   Column     Dtype   
---  ------     -----   
 0   ID_graph   int64   
 1   stnode     int64   
 2   endnode    int64   
 3   time       float64 
 4   length     float64 
 5   highway    object  
 6   osmid      int64   
 7   geometry   geometry
 8   PFU_1in10  float64 
dtypes: float64(3), geometry(1), int64(4), object(1)
memory usage: 314.0+ MB


In [24]:
join10 = join10.to_crs("EPSG:4326")
join10.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [25]:
join10

,ID_graph,stnode,endnode,time,length,highway,osmid,geometry,PFU_1in10
0,0,358284990,5217543379,2.385144,33.127,unclassified,59618174,"LINESTRING (-12.32347 12.38119, -12.32368 12.3...",NaN
1,1,358284990,1888282175,0.769920,12.832,tertiary,178482063,"LINESTRING (-12.32347 12.38119, -12.32351 12.3...",NaN
2,2,358284990,5329792467,2.926860,48.781,tertiary,178482063,"LINESTRING (-12.32347 12.38119, -12.32317 12.3...",NaN
3,3,358284993,1888282575,7.594620,126.577,tertiary,178470940,"LINESTRING (-12.28135 12.41380, -12.28205 12.4...",NaN
4,4,358284993,1888198886,7.234680,120.578,tertiary,178470940,"LINESTRING (-12.28135 12.41380, -12.28061 12.4...",NaN
...,...,...,...,...,...,...,...,...,...
4012560,4012560,9246539941,9246539942,6.906780,76.742,residential,366052716,"LINESTRING (-17.48521 14.72445, -17.48451 14.7...",NaN
4012561,4012561,9246539942,9246539941,6.906780,76.742,residential,366052716,"LINESTRING (-17.48451 14.72456, -17.48521 14.7...",NaN
4012562,4012562,9246539942,3700438702,0.076140,0.846,residential,366052716,"LINESTRING (-17.48451 14.72456, -17.48451 14.7...",NaN
4012563,4012563,9276108905,6048975958,8.840674,171.902,secondary,177950649,"LINESTRING (-17.50499 14.74971, -17.50357 14.7...",NaN


In [26]:
# Fewer errors farther down when using dataframe instead of gdf
join10_df = pd.DataFrame(join10)
join10_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4115534 entries, 0 to 4012564
Data columns (total 9 columns):
 #   Column     Dtype   
---  ------     -----   
 0   ID_graph   int64   
 1   stnode     int64   
 2   endnode    int64   
 3   time       float64 
 4   length     float64 
 5   highway    object  
 6   osmid      int64   
 7   geometry   geometry
 8   PFU_1in10  float64 
dtypes: float64(3), geometry(1), int64(4), object(1)
memory usage: 314.0+ MB


In [27]:
join10_df

,ID_graph,stnode,endnode,time,length,highway,osmid,geometry,PFU_1in10
0,0,358284990,5217543379,2.385144,33.127,unclassified,59618174,"LINESTRING (-12.32347 12.38119, -12.32368 12.3...",NaN
1,1,358284990,1888282175,0.769920,12.832,tertiary,178482063,"LINESTRING (-12.32347 12.38119, -12.32351 12.3...",NaN
2,2,358284990,5329792467,2.926860,48.781,tertiary,178482063,"LINESTRING (-12.32347 12.38119, -12.32317 12.3...",NaN
3,3,358284993,1888282575,7.594620,126.577,tertiary,178470940,"LINESTRING (-12.28135 12.41380, -12.28205 12.4...",NaN
4,4,358284993,1888198886,7.234680,120.578,tertiary,178470940,"LINESTRING (-12.28135 12.41380, -12.28061 12.4...",NaN
...,...,...,...,...,...,...,...,...,...
4012560,4012560,9246539941,9246539942,6.906780,76.742,residential,366052716,"LINESTRING (-17.48521 14.72445, -17.48451 14.7...",NaN
4012561,4012561,9246539942,9246539941,6.906780,76.742,residential,366052716,"LINESTRING (-17.48451 14.72456, -17.48521 14.7...",NaN
4012562,4012562,9246539942,3700438702,0.076140,0.846,residential,366052716,"LINESTRING (-17.48451 14.72456, -17.48451 14.7...",NaN
4012563,4012563,9276108905,6048975958,8.840674,171.902,secondary,177950649,"LINESTRING (-17.50499 14.74971, -17.50357 14.7...",NaN


In [29]:
join10_df.to_csv(os.path.join(out_pth, 'gTime_flood_intermediate.csv'))

### Create speed penalties.
Note: Flood depths are in centimeters. FATHOM uses meters, but conversion process to vector required some finessing. 

In [31]:
# Give a depth to the nodes that don't cross a flood point. 
join10.loc[join10['PFU_1in10'].isnull(), 'PFU_1in10'] = -1

In [32]:
join10["t10"] = 1 # This is the penalty column.
join10.loc[join10['PFU_1in10'] < 0, 't10'] = 1 # Where no flood crosses, keep the default value (no penalty).
join10.loc[(join10['PFU_1in10'] > 10) & (join10['PFU_1in10'] <= 30), 't10'] = 1.25
join10.loc[(join10['PFU_1in10'] > 30) & (join10['PFU_1in10'] <= 60), 't10'] = 2
join10.loc[(join10['PFU_1in10'] > 60) & (join10['PFU_1in10'] <= 90), 't10'] = 5
join10.loc[(join10['PFU_1in10'] > 90), 't10'] = 9999
join10

,ID_graph,stnode,endnode,time,length,highway,osmid,geometry,PFU_1in10,t10
0,0,358284990,5217543379,2.385144,33.127,unclassified,59618174,"LINESTRING (-12.32347 12.38119, -12.32368 12.3...",-1.0,1.0
1,1,358284990,1888282175,0.769920,12.832,tertiary,178482063,"LINESTRING (-12.32347 12.38119, -12.32351 12.3...",-1.0,1.0
2,2,358284990,5329792467,2.926860,48.781,tertiary,178482063,"LINESTRING (-12.32347 12.38119, -12.32317 12.3...",-1.0,1.0
3,3,358284993,1888282575,7.594620,126.577,tertiary,178470940,"LINESTRING (-12.28135 12.41380, -12.28205 12.4...",-1.0,1.0
4,4,358284993,1888198886,7.234680,120.578,tertiary,178470940,"LINESTRING (-12.28135 12.41380, -12.28061 12.4...",-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
4012560,4012560,9246539941,9246539942,6.906780,76.742,residential,366052716,"LINESTRING (-17.48521 14.72445, -17.48451 14.7...",-1.0,1.0
4012561,4012561,9246539942,9246539941,6.906780,76.742,residential,366052716,"LINESTRING (-17.48451 14.72456, -17.48521 14.7...",-1.0,1.0
4012562,4012562,9246539942,3700438702,0.076140,0.846,residential,366052716,"LINESTRING (-17.48451 14.72456, -17.48451 14.7...",-1.0,1.0
4012563,4012563,9276108905,6048975958,8.840674,171.902,secondary,177950649,"LINESTRING (-17.50499 14.74971, -17.50357 14.7...",-1.0,1.0


In [33]:
# Turn the penalty column into a flood-affected time column.
join10['t10'] = join10['t10'] * join10['time']
join10

,ID_graph,stnode,endnode,time,length,highway,osmid,geometry,PFU_1in10,t10
0,0,358284990,5217543379,2.385144,33.127,unclassified,59618174,"LINESTRING (-12.32347 12.38119, -12.32368 12.3...",-1.0,2.385144
1,1,358284990,1888282175,0.769920,12.832,tertiary,178482063,"LINESTRING (-12.32347 12.38119, -12.32351 12.3...",-1.0,0.769920
2,2,358284990,5329792467,2.926860,48.781,tertiary,178482063,"LINESTRING (-12.32347 12.38119, -12.32317 12.3...",-1.0,2.926860
3,3,358284993,1888282575,7.594620,126.577,tertiary,178470940,"LINESTRING (-12.28135 12.41380, -12.28205 12.4...",-1.0,7.594620
4,4,358284993,1888198886,7.234680,120.578,tertiary,178470940,"LINESTRING (-12.28135 12.41380, -12.28061 12.4...",-1.0,7.234680
...,...,...,...,...,...,...,...,...,...,...
4012560,4012560,9246539941,9246539942,6.906780,76.742,residential,366052716,"LINESTRING (-17.48521 14.72445, -17.48451 14.7...",-1.0,6.906780
4012561,4012561,9246539942,9246539941,6.906780,76.742,residential,366052716,"LINESTRING (-17.48451 14.72456, -17.48521 14.7...",-1.0,6.906780
4012562,4012562,9246539942,3700438702,0.076140,0.846,residential,366052716,"LINESTRING (-17.48451 14.72456, -17.48451 14.7...",-1.0,0.076140
4012563,4012563,9276108905,6048975958,8.840674,171.902,secondary,177950649,"LINESTRING (-17.50499 14.74971, -17.50357 14.7...",-1.0,8.840674


In [34]:
join10.to_csv(os.path.join(out_pth, 'join10.csv'))

### Convert back to graph object.

In [36]:
print('start: %s\n' % time.ctime())
G_flood = gn.edges_and_nodes_gdf_to_graph(gTime_node, join10, node_tag='node_ID', u_tag='stnode', v_tag='endnode', geometry_tag='geometry')
gn.example_edge(G_flood10, 10)
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

(358284990, 5217543379, {'geometry': <shapely.geometry.linestring.LineString object at 0x000001A944E8E1F0>, 'ID_graph': 0, 'time': 2.3851440000000004, 'length': 33.127, 'highway': 'unclassified', 'osmid': 59618174, 'PFU_1in10': -1.0, 't10': 2.3851440000000004})
(358284990, 1888282175, {'geometry': <shapely.geometry.linestring.LineString object at 0x000001A944E8E250>, 'ID_graph': 1, 'time': 0.76992, 'length': 12.832, 'highway': 'tertiary', 'osmid': 178482063, 'PFU_1in10': -1.0, 't10': 0.76992})
(358284990, 5329792467, {'geometry': <shapely.geometry.linestring.LineString object at 0x000001A944E8E2B0>, 'ID_graph': 2, 'time': 2.92686, 'length': 48.781, 'highway': 'tertiary', 'osmid': 178482063, 'PFU_1in10': -1.0, 't10': 2.92686})
(5217543379, 358284990, {'geometry': <shapely.geometry.linestring.LineString object at 0x000001A945945340>, 'ID_graph': 16698, 'time': 2.3851440000000004, 'length': 33.127, 'highway': 'unclassified', 'osmid': 59618174, 'PFU_1in10': -1.0, 't10': 2.3851440000000004}

In [38]:
print('start: %s\n' % time.ctime())
gn.save(G_flood, 'gTime_flood10', pth, edges = True, nodes = True)
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Fri Dec  3 15:49:34 2021


end: Fri Dec  3 16:04:21 2021

--- processing complete


### Create travel time values for the road nodes nearest to each service.

Using calculate_OD.

In [39]:
# If starting a new session, load from file.
HDurban_snap = os.path.join(out_pth, "HDurban_snap.csv")
HDurban_snap = pd.read_csv(HDurban_snap)
hamlet_snap = os.path.join(out_pth, "hamlet_snap.csv")
hamlet_snap = pd.read_csv(hamlet_snap)
ag_snap = os.path.join(out_pth, "ag_snap.csv")
ag_snap = pd.read_csv(ag_snap)

In [ ]:
G_flood = nx.read_gpickle("SEN-Cdrive/outputs/gTime_flood10.pickle")

In [40]:
# We only need to find the origin-destination pairs for nodes closest to the origins and services,
# and some nodes will be the nearest for more than one service (and definitely for multiple origins).
list_hamlet = list(hamlet_snap.NN.unique())
list_ag = list(ag_snap.NN.unique())
originslist = list_hamlet + list_ag
origins = list(set(originslist))

In [41]:
dests = list(HDurban_snap.NN.unique()) 

In [42]:
len(origins)

554159

In [43]:
len(dests) 

58

In [44]:
fail_value = 999999999 # If there is no shortest path, the OD pair will be assigned the fail value.

In [46]:
print('start: %s\n' % time.ctime())
OD = gn.calculate_OD(G_flood, origins, dests, fail_value, weight = 't10')
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Fri Dec  3 16:09:00 2021


end: Fri Dec  3 16:29:33 2021

--- processing complete


In [47]:
OD_df = pd.DataFrame(OD, index = origins, columns = dests)

In [48]:
OD_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 554159 entries, 5755633676 to 6024069080
Data columns (total 58 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   6058226279  554159 non-null  float64
 1   6029307183  554159 non-null  float64
 2   4998093094  554159 non-null  float64
 3   2201506815  554159 non-null  float64
 4   3474499811  554159 non-null  float64
 5   1697006012  554159 non-null  float64
 6   1901689169  554159 non-null  float64
 7   6032060028  554159 non-null  float64
 8   6040927878  554159 non-null  float64
 9   3449495495  554159 non-null  float64
 10  3990543961  554159 non-null  float64
 11  8972391475  554159 non-null  float64
 12  3418418812  554159 non-null  float64
 13  1983641803  554159 non-null  float64
 14  6014451367  554159 non-null  float64
 15  6027163276  554159 non-null  float64
 16  2833577858  554159 non-null  float64
 17  4656728818  554159 non-null  float64
 18  6045659373  554159 non-null  fl

In [49]:
# Convert to minutes and save to file.
OD_min = OD_df[OD_df <fail_value] / 60
OD_min.to_csv(os.path.join(out_pth, 'OD_flood10_allorigins.csv'))
OD_min

,6058226279,6029307183,4998093094,2201506815,3474499811,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,1936967272,3496518021,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
5755633676,164.170367,126.126118,184.282364,99.701417,80.802523,217.099791,230.174294,214.096126,148.241593,182.276582,...,271.411917,324.415053,252.311344,252.128015,188.615582,227.014789,227.933200,233.256238,180.066383,192.563357
5755633677,164.198257,126.154007,184.310253,99.729306,80.830413,217.127680,230.202183,214.124016,148.269482,182.304471,...,271.439806,324.442942,252.339233,252.155904,188.643471,227.042679,227.961089,233.284127,180.094272,192.591246
5755633678,164.250489,126.206239,184.362485,99.781538,80.882645,217.179912,230.254416,214.176248,148.321714,182.356704,...,271.492039,324.495174,252.391465,252.208137,188.695703,227.094911,228.013322,233.336359,180.146505,192.643478
5755633680,164.298646,126.254397,184.410642,99.829695,80.930802,217.228069,230.302573,214.224405,148.369871,182.404861,...,271.540196,324.543331,252.439622,252.256294,188.743860,227.143068,228.061479,233.384516,180.194662,192.691635
5755633681,164.351966,126.307716,184.463962,99.883015,80.984122,217.281389,230.355893,214.277725,148.423191,182.458180,...,271.593516,324.596651,252.492942,252.309614,188.797180,227.196388,228.114799,233.437836,180.247981,192.744955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3766810947,379.245050,222.092873,462.782345,378.201398,361.348829,495.599772,508.674275,492.596107,426.741574,460.776563,...,549.911898,602.915034,133.884423,133.701095,467.115563,505.514771,506.433181,511.756219,458.566364,471.063338
6024069077,314.537140,276.492890,85.696724,72.120437,159.823245,54.037227,93.524982,51.033562,27.081243,61.920352,...,106.916860,159.919995,402.678116,402.494787,90.029942,88.583631,88.055367,87.967827,59.710153,72.207126
6024069078,314.512741,276.468492,85.672325,72.096038,159.798846,54.012828,93.500584,51.009164,27.056845,61.895953,...,106.892462,159.895597,402.653718,402.470389,90.005543,88.559233,88.030969,87.943429,59.685754,72.182728
6024069079,314.399846,276.355597,85.559430,71.983144,159.685951,53.899933,93.387689,50.896269,26.943950,61.783059,...,106.779567,159.782702,402.540823,402.357494,89.892649,88.446338,87.918074,87.830534,59.572860,72.069833


In [50]:
# Create origin-specific matrix and save to file.
OD_ag = OD_df.loc[list_ag,: ]
OD_ag = OD_ag[OD_ag < fail_value] / 60 
OD_ag.to_csv(os.path.join(out_pth, 'OD_flood10_ag.csv'))
OD_ag

,6058226279,6029307183,4998093094,2201506815,3474499811,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,1936967272,3496518021,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
3656617769,36265.165894,36227.121644,35985.888199,36022.749191,36110.451999,35950.022875,35988.491079,35933.074908,35977.709998,35945.457559,...,35857.072771,35796.110245,36304.824256,36304.466244,35987.404936,35980.315905,35979.426026,35979.319249,35947.558798,35948.564916
3656617804,36265.190115,36227.145865,35985.912420,36022.773412,36110.476220,35950.047096,35988.515300,35933.099129,35977.734219,35945.481780,...,35857.096992,35796.134466,36304.848477,36304.490465,35987.429157,35980.340126,35979.450247,35979.343470,35947.583019,35948.589137
3656618066,36265.275939,36227.231689,35985.998244,36022.859236,36110.562044,35950.132920,35988.601124,35933.184953,35977.820043,35945.567604,...,35857.182816,35796.220290,36304.934301,36304.576289,35987.514981,35980.425950,35979.536071,35979.429294,35947.668843,35948.674961
9208195325,95918.815749,95880.771500,95639.538054,95676.399047,95764.101854,95603.672730,95642.140934,95586.724764,95631.359853,95599.107414,...,95510.722626,95449.760100,95958.474111,95958.116099,95641.054792,95633.965760,95633.075882,95632.969104,95601.208653,95602.214771
9208195303,95923.116975,95885.072726,95643.839280,95680.700273,95768.403080,95607.973956,95646.442160,95591.025990,95635.661079,95603.408640,...,95515.023852,95454.061326,95962.775337,95962.417325,95645.356018,95638.266986,95637.377108,95637.270330,95605.509879,95606.515997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8585063327,152.061298,107.072670,396.421053,311.840106,294.987538,429.238480,442.312984,426.234816,360.380282,394.415272,...,483.550607,536.553742,292.691721,292.508393,400.754271,439.153479,440.071890,445.394927,392.205073,404.702046
8585142396,182.025845,89.230105,378.578488,293.997541,277.144972,411.395915,424.470418,408.392250,342.537717,376.572706,...,465.708041,518.711177,257.995618,257.812289,382.911706,421.310913,422.229324,427.552362,374.362507,386.859481
4618077944,434.015871,290.324008,532.001977,447.421030,430.568461,564.819404,577.893907,561.815739,495.961206,529.996195,...,619.131530,672.134665,203.104055,202.920726,536.335195,574.734402,575.652813,580.975851,527.785996,540.282969
4618077951,433.410431,289.718567,531.396536,446.815589,429.963020,564.213963,577.288466,561.210298,495.355765,529.390754,...,618.526089,671.529225,202.498614,202.315286,535.729754,574.128962,575.047372,580.370410,527.180555,539.677529


In [51]:
OD_hamlet = OD_df.loc[list_hamlet,: ]
OD_hamlet = OD_hamlet[OD_hamlet < fail_value] / 60 
OD_hamlet.to_csv(os.path.join(out_pth, 'OD_flood10_hamlet.csv'))
OD_hamlet

,6058226279,6029307183,4998093094,2201506815,3474499811,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,1936967272,3496518021,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
7761872870,62.891073,245.798986,389.498891,304.917944,288.065376,422.316318,435.390822,419.312654,353.458120,387.493110,...,476.628445,529.631580,426.887347,426.704018,393.832109,432.231317,433.149728,438.472765,385.282911,397.779884
7761872869,62.872513,245.780427,389.480332,304.899385,288.046817,422.297759,435.372262,419.294095,353.439561,387.474550,...,476.609885,529.613021,426.868787,426.685459,393.813550,432.212758,433.131168,438.454206,385.264351,397.761325
6442044321,61.862681,244.770595,388.470500,303.889553,287.036985,421.287927,434.362430,418.284263,352.429729,386.464718,...,475.600053,528.603189,425.858955,425.675627,392.803718,431.202926,432.121336,437.444374,384.254519,396.751493
2142496418,63.316203,246.224117,389.924022,305.343075,288.490507,422.741449,435.815952,419.737785,353.883251,387.918240,...,477.053575,530.056711,427.312477,427.129149,394.257240,432.656448,433.574858,438.897896,385.708041,398.205015
2142496429,63.698577,246.606491,390.306396,305.725449,288.872881,423.123823,436.198326,420.120159,354.265625,388.300614,...,477.435949,530.439085,427.694851,427.511523,394.639614,433.038822,433.957232,439.280270,386.090415,398.587389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9207762346,590.010067,538.743620,392.316477,344.627112,435.296172,372.302310,411.790066,365.139842,297.697339,262.766567,...,284.677977,244.782814,348.428662,348.070650,396.649695,406.848714,406.320451,406.232911,272.057049,265.128454
8463584916,495.475639,457.431389,221.796126,250.092684,340.761744,185.930802,224.399006,168.982836,203.162911,169.766119,...,91.112914,64.552430,505.146696,504.788684,223.312864,219.457655,218.929391,218.841851,171.867358,172.873476
8463593882,499.234844,461.190594,225.764336,253.851889,344.520949,189.899012,228.367216,172.951045,206.922116,173.525324,...,94.872119,68.659633,492.534686,492.176674,227.281073,223.425864,222.897601,222.810061,175.626563,176.632681
9208004175,95942.692657,95904.648407,95663.414961,95700.275954,95787.978761,95627.549637,95666.017842,95610.601671,95655.236760,95622.984321,...,95534.599533,95473.637007,95982.351018,95981.993006,95664.931699,95657.842667,95656.952789,95656.846011,95625.085560,95626.091678


### Filter 1st nearest

#### Check each file to make sure nearest neighbor column is named correctly. If not, rename.

In [52]:
# Reload from file even if already loaded. Quickest way to ensure NN is a column rather than only the index.
OD_hamlet = os.path.join(out_pth, "OD_flood10_hamlet.csv")
OD_hamlet = pd.read_csv(OD_hamlet)
OD_ag = os.path.join(out_pth, "OD_flood10_ag.csv")
OD_ag = pd.read_csv(OD_ag)

In [53]:
OD_hamlet

,Unnamed: 0,6058226279,6029307183,4998093094,2201506815,3474499811,1697006012,1901689169,6032060028,6040927878,...,1968458114,1936967272,3496518021,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
0,7761872870,62.891073,245.798986,389.498891,304.917944,288.065376,422.316318,435.390822,419.312654,353.458120,...,476.628445,529.631580,426.887347,426.704018,393.832109,432.231317,433.149728,438.472765,385.282911,397.779884
1,7761872869,62.872513,245.780427,389.480332,304.899385,288.046817,422.297759,435.372262,419.294095,353.439561,...,476.609885,529.613021,426.868787,426.685459,393.813550,432.212758,433.131168,438.454206,385.264351,397.761325
2,6442044321,61.862681,244.770595,388.470500,303.889553,287.036985,421.287927,434.362430,418.284263,352.429729,...,475.600053,528.603189,425.858955,425.675627,392.803718,431.202926,432.121336,437.444374,384.254519,396.751493
3,2142496418,63.316203,246.224117,389.924022,305.343075,288.490507,422.741449,435.815952,419.737785,353.883251,...,477.053575,530.056711,427.312477,427.129149,394.257240,432.656448,433.574858,438.897896,385.708041,398.205015
4,2142496429,63.698577,246.606491,390.306396,305.725449,288.872881,423.123823,436.198326,420.120159,354.265625,...,477.435949,530.439085,427.694851,427.511523,394.639614,433.038822,433.957232,439.280270,386.090415,398.587389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61225,9207762346,590.010067,538.743620,392.316477,344.627112,435.296172,372.302310,411.790066,365.139842,297.697339,...,284.677977,244.782814,348.428662,348.070650,396.649695,406.848714,406.320451,406.232911,272.057049,265.128454
61226,8463584916,495.475639,457.431389,221.796126,250.092684,340.761744,185.930802,224.399006,168.982836,203.162911,...,91.112914,64.552430,505.146696,504.788684,223.312864,219.457655,218.929391,218.841851,171.867358,172.873476
61227,8463593882,499.234844,461.190594,225.764336,253.851889,344.520949,189.899012,228.367216,172.951045,206.922116,...,94.872119,68.659633,492.534686,492.176674,227.281073,223.425864,222.897601,222.810061,175.626563,176.632681
61228,9208004175,95942.692657,95904.648407,95663.414961,95700.275954,95787.978761,95627.549637,95666.017842,95610.601671,95655.236760,...,95534.599533,95473.637007,95982.351018,95981.993006,95664.931699,95657.842667,95656.952789,95656.846011,95625.085560,95626.091678


In [54]:
OD_ag.rename(columns={'Unnamed: 0': 'NN'}, inplace=True) 
OD_hamlet.rename(columns={'Unnamed: 0': 'NN'}, inplace=True) 
OD_ag

,NN,6058226279,6029307183,4998093094,2201506815,3474499811,1697006012,1901689169,6032060028,6040927878,...,1968458114,1936967272,3496518021,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
0,3656617769,36265.165894,36227.121644,35985.888199,36022.749191,36110.451999,35950.022875,35988.491079,35933.074908,35977.709998,...,35857.072771,35796.110245,36304.824256,36304.466244,35987.404936,35980.315905,35979.426026,35979.319249,35947.558798,35948.564916
1,3656617804,36265.190115,36227.145865,35985.912420,36022.773412,36110.476220,35950.047096,35988.515300,35933.099129,35977.734219,...,35857.096992,35796.134466,36304.848477,36304.490465,35987.429157,35980.340126,35979.450247,35979.343470,35947.583019,35948.589137
2,3656618066,36265.275939,36227.231689,35985.998244,36022.859236,36110.562044,35950.132920,35988.601124,35933.184953,35977.820043,...,35857.182816,35796.220290,36304.934301,36304.576289,35987.514981,35980.425950,35979.536071,35979.429294,35947.668843,35948.674961
3,9208195325,95918.815749,95880.771500,95639.538054,95676.399047,95764.101854,95603.672730,95642.140934,95586.724764,95631.359853,...,95510.722626,95449.760100,95958.474111,95958.116099,95641.054792,95633.965760,95633.075882,95632.969104,95601.208653,95602.214771
4,9208195303,95923.116975,95885.072726,95643.839280,95680.700273,95768.403080,95607.973956,95646.442160,95591.025990,95635.661079,...,95515.023852,95454.061326,95962.775337,95962.417325,95645.356018,95638.266986,95637.377108,95637.270330,95605.509879,95606.515997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541456,8585063327,152.061298,107.072670,396.421053,311.840106,294.987538,429.238480,442.312984,426.234816,360.380282,...,483.550607,536.553742,292.691721,292.508393,400.754271,439.153479,440.071890,445.394927,392.205073,404.702046
541457,8585142396,182.025845,89.230105,378.578488,293.997541,277.144972,411.395915,424.470418,408.392250,342.537717,...,465.708041,518.711177,257.995618,257.812289,382.911706,421.310913,422.229324,427.552362,374.362507,386.859481
541458,4618077944,434.015871,290.324008,532.001977,447.421030,430.568461,564.819404,577.893907,561.815739,495.961206,...,619.131530,672.134665,203.104055,202.920726,536.335195,574.734402,575.652813,580.975851,527.785996,540.282969
541459,4618077951,433.410431,289.718567,531.396536,446.815589,429.963020,564.213963,577.288466,561.210298,495.355765,...,618.526089,671.529225,202.498614,202.315286,535.729754,574.128962,575.047372,580.370410,527.180555,539.677529


#### Find first, second, and third nearest destination for each origin node. 

In [55]:
fail_value = 999999999

In [56]:
# Nearest
OD_ag["ag_HD1F10"] = 0
sub = OD_ag.iloc[:,1:-1] # Filtering out the newly created field and the node ID column. ("include everything between column 0 and the last column")
OD_ag["ag_HD1F10"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
ag1 = OD_ag[['NN', 'ag_HD1F10']] # Remove unnecessary OD values.


# Second nearest
dupes = OD_ag.apply(pd.Series.duplicated, axis = 1, keep=False) # If a number is repeated within a row, value is True. If not, False.
# The first time this is done, there should be two True values per row, unless any POIs are equidistant.
dupes = OD_ag.where(~dupes, fail_value) # For any value that appears more than once in its row, it is replaced with the fail_value.
OD_ag["ag_HD2F10"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column. No need to filter 1st nearest as its new "dupes" value is too high to be caught.
OD_ag["ag_HD2F10"] = Dsub.min(axis=1) 
ag2 = OD_ag.loc[:,['NN', 'ag_HD2F10']] 


# Third nearest
dupes = OD_ag.apply(pd.Series.duplicated, axis = 1, keep=False)
# Since this includes both first and second nearest columns, there should be four True values per row, unless POIs are equidistant.
dupes = OD_ag.where(~dupes, fail_value)

OD_ag["ag_HD3F10"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column.
OD_ag["ag_HD3F10"] = Dsub.min(axis=1)
ag3 = OD_ag.loc[:,['NN', 'ag_HD3F10']]

# Combine and write to file
ag_all = OD_ag.loc[:,['NN', 'ag_HD1F10', 'ag_HD2F10', 'ag_HD3F10']]
ag_all.to_csv(os.path.join(out_pth, 'ag_to_HDurban_flood10.csv'))
ag_all.head()

,NN,ag_HD1F10,ag_HD2F10,ag_HD3F10
0,3656617769,35796.110245,35797.064528,35802.748075
1,3656617804,35796.134466,35797.088749,35802.772296
2,3656618066,35796.220290,35797.174573,35802.858120
3,9208195325,95449.760100,95450.714384,95456.397931
4,9208195303,95454.061326,95455.015610,95460.699157


In [57]:
# Nearest
OD_hamlet["ha_HD1F10"] = 0
sub = OD_hamlet.iloc[:,1:-1] # Filtering out the newly created field and the node ID column. ("include everything between column 0 and the last column")
OD_hamlet["ha_HD1F10"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
hamlet1 = OD_hamlet[['NN', 'ha_HD1F10']] # Remove unnecessary OD values.


# Second nearest
dupes = OD_hamlet.apply(pd.Series.duplicated, axis = 1, keep=False) # If a number is repeated within a row, value is True. If not, False.
# The first time this is done, there should be two True values per row, unless any POIs are equidistant.
dupes = OD_hamlet.where(~dupes, fail_value) # For any value that appears more than once in its row, it is replaced with the fail_value.
OD_hamlet["ha_HD2F10"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column. No need to filter 1st nearest as its new "dupes" value is too high to be caught.
OD_hamlet["ha_HD2F10"] = Dsub.min(axis=1) 
hamlet2 = OD_hamlet.loc[:,['NN', 'ha_HD2F10']] 


# Third nearest
dupes = OD_hamlet.apply(pd.Series.duplicated, axis = 1, keep=False)
# Since this includes both first and second nearest columns, there should be four True values per row, unless POIs are equidistant.
dupes = OD_hamlet.where(~dupes, fail_value)
OD_hamlet["ha_HD3F10"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column.
OD_hamlet["ha_HD3F10"] = Dsub.min(axis=1)
hamlet3 = OD_hamlet.loc[:,['NN', 'ha_HD3F10']]


# Combine and write to file
hamlet_all = OD_hamlet.loc[:,['NN', 'ha_HD1F10', 'ha_HD2F10', 'ha_HD3F10']]
hamlet_all.to_csv(os.path.join(out_pth, 'hamlet_to_HDurban_flood10.csv'))
hamlet_all.head()

,NN,ha_HD1F10,ha_HD2F10,ha_HD3F10
0,7761872870,62.891073,66.259381,245.798986
1,7761872869,62.872513,66.240822,245.780427
2,6442044321,61.862681,65.230990,244.770595
3,2142496418,63.316203,66.684512,246.224117
4,2142496429,63.698577,67.066886,246.606491


### Join back to georeferenced _snap file.

In [13]:
# If starting new session, reload from file.
ag_all = os.path.join(out_pth, "ag_to_HDurban_flood10.csv")
ag_all = pd.read_csv(ag_all)
ag_all.info() # Check to make sure NN data type matches its corresponding _snap file

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541461 entries, 0 to 541460
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  541461 non-null  int64  
 1   NN          541461 non-null  int64  
 2   ag_HD1F10   541453 non-null  float64
 3   ag_HD2F10   541461 non-null  float64
 4   ag_HD3F10   541461 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 20.7 MB


In [ ]:
hamlet_all = os.path.join(out_pth, "hamlet_to_HDurban_flood10.csv")
hamlet_all = pd.read_csv(hamlet_all)
hamlet_all.info()

In [10]:
ag_snap = os.path.join(out_pth, "ag_snap.csv")
ag_snap = pd.read_csv(ag_snap)
ag_snap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6852701 entries, 0 to 6852700
Data columns (total 6 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   ID          int64  
 2   LC_90m      int64  
 3   geometry    object 
 4   NN          int64  
 5   NN_dist     float64
dtypes: float64(1), int64(4), object(1)
memory usage: 313.7+ MB


In [ ]:
hamlet_snap = os.path.join(out_pth, "hamlet_snap.csv")
hamlet_snap = pd.read_csv(hamlet_snap)
hamlet_snap.info()

In [14]:
ag_to_HDurban = pd.merge(ag_snap, ag_all, on='NN',how='left')
ag_to_HDurban

,Unnamed: 0_x,ID,LC_90m,geometry,NN,NN_dist,Unnamed: 0_y,ag_HD1F10,ag_HD2F10,ag_HD3F10
0,0,17,1700,POINT (-15.549972323895453 16.693437843999725),3656617769,6542.678694,0,35796.110245,35797.064528,35802.748075
1,1,18,1700,POINT (-15.54916384014042 16.693437844000037),3656617769,6511.929511,0,35796.110245,35797.064528,35802.748075
2,2,19,1700,POINT (-15.54835535638528 16.693437844000353),3656617769,6482.181173,0,35796.110245,35797.064528,35802.748075
3,3,20,1700,POINT (-15.547546872630239 16.69343784400067),3656617769,6453.447521,0,35796.110245,35797.064528,35802.748075
4,4,21,1700,POINT (-15.5467383888751 16.693437844000975),3656617804,6425.455616,1,35796.134466,35797.088749,35802.772296
...,...,...,...,...,...,...,...,...,...,...
6852696,6852696,7090377,1700,POINT (-13.000010011573373 12.307410002750597),4618077945,887.864052,540807,197.596792,198.800544,202.839741
6852697,6852697,7090378,1700,POINT (-12.99112001158105 12.307410002752631),4618077951,1645.424404,541459,197.072337,198.276089,202.315286
6852698,6852698,7090379,1700,POINT (-12.856100011697764 12.307410002783556),4818730342,2863.665090,540909,200.274858,201.478610,205.517807
6852699,6852699,7090380,1700,POINT (-12.295830012184553 12.307410002913299),2461705282,104.777301,540999,310.907583,312.111335,316.150531


In [59]:
hamlet_to_HDurban = pd.merge(hamlet_snap, hamlet_all, on='NN',how='left')
hamlet_to_HDurban

,Unnamed: 0,Unnamed_ 0,mgrs_code,type,GlobalID,Shape_Leng,Shape_Area,geometry,NN,NN_dist,ha_HD1F10,ha_HD2F10,ha_HD3F10
0,0,0,28PCU1265_01,hamlet,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},0.004314,0.000001,POINT (-16.721473282454415 12.348636090165247),7761872870,307.058089,62.891073,66.259381,245.798986
1,1,1,28PCU1365_01,hamlet,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},0.009910,0.000006,POINT (-16.716456507935607 12.34788723564788),7761872869,801.450257,62.872513,66.240822,245.780427
2,2,2,28PCU1365_02,hamlet,{D03C2B85-5F35-4EE8-8346-B83494628F26},0.003754,0.000001,POINT (-16.713855008830738 12.350880992129111),6442044321,694.273717,61.862681,65.230990,244.770595
3,3,3,28PCU1566_01,hamlet,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},0.004401,0.000002,POINT (-16.701275174874546 12.355585269999269),2142496418,689.246791,63.316203,66.684512,246.224117
4,4,4,28PCU1566_02,hamlet,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},0.005357,0.000002,POINT (-16.698773736706396 12.356804484409668),2142496429,607.912599,63.698577,67.066886,246.606491
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125881,125881,125881,28QED6412_03,hamlet,{5555A010-36B2-47D2-96C4-BDD1E59111ED},0.005397,0.000002,POINT (-14.397827933065358 16.394142941310925),8592243241,5089.086102,186.960455,189.048692,195.107512
125882,125882,125882,28QED6413_03,hamlet,{20205A44-8B9D-4FCE-B14C-53826594DB5A},0.003610,0.000001,POINT (-14.397473236932905 16.405676938062538),6375187769,3949.180081,185.031023,187.119261,193.178080
125883,125883,125883,28QED6413_04,hamlet,{AC6A169C-FD0E-4DF6-BDAB-B69FBD04BFAF},0.015471,0.000008,POINT (-14.400364192239715 16.404303134272737),8592243457,4008.851333,186.661697,188.749934,194.808753
125884,125884,125884,28QED6424_03,hamlet,{51593C65-B268-4BA1-8212-E43232C021FF},0.003883,0.000001,POINT (-14.396243884215123 16.49715066208162),3646207611,1826.174300,182.868457,184.956694,191.015513


In [ ]:
hamlet_to_HDurban.to_csv(os.path.join(out_pth, 'hamlet_to_HDurban_flood10.csv'))
ag_to_HDurban.to_csv(os.path.join(out_pth, 'ag_to_HDurban_flood10.csv'))

The geometry column is missing a comma for some reason and isn't reading as a GDF in Python or Arc. So let's quick make a shapefile version by extracting the points. Can use this as well for the non-flood travel times, which were saved as csv.

In [ ]:
hamlet_to_HDurban = os.path.join(out_pth, "hamlet_to_HDurban.csv")
hamlet_to_HDurban = pd.read_csv(hamlet_to_HDurban)

In [60]:
hamlet_to_HDurban["geometry"] = hamlet_to_HDurban["geometry"].astype('str')
hamlet_to_HDurban["geometry"]  = hamlet_to_HDurban["geometry"] .str.strip('POINT ')
hamlet_to_HDurban["geometry"]  = hamlet_to_HDurban["geometry"] .str.strip('()')
XY = hamlet_to_HDurban["geometry"] .str.split(" ", expand=True)
hamlet_to_HDurban["X"] = XY[0]
hamlet_to_HDurban["Y"] = XY[1]
hamlet_to_HDurban["X"] = hamlet_to_HDurban["X"].astype('float')
hamlet_to_HDurban["Y"] = hamlet_to_HDurban["Y"].astype('float')
hamlet_to_HDurban = hamlet_to_HDurban.drop(columns=['geometry'])
hamlet_to_HDurban.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125886 entries, 0 to 125885
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  125886 non-null  int64  
 1   Unnamed_ 0  125886 non-null  int64  
 2   mgrs_code   125886 non-null  object 
 3   type        125886 non-null  object 
 4   GlobalID    125886 non-null  object 
 5   Shape_Leng  125886 non-null  float64
 6   Shape_Area  125886 non-null  float64
 7   NN          125886 non-null  int64  
 8   NN_dist     125886 non-null  float64
 9   ha_HD1F10   125883 non-null  float64
 10  ha_HD2F10   125886 non-null  float64
 11  ha_HD3F10   125886 non-null  float64
 12  X           125886 non-null  float64
 13  Y           125886 non-null  float64
dtypes: float64(8), int64(3), object(3)
memory usage: 14.4+ MB


In [9]:
ag_to_HDurban = os.path.join(out_pth, "ag_to_HDurban_flood10.csv")
ag_to_HDurban = pd.read_csv(ag_to_HDurban)

In [15]:
ag_to_HDurban["geometry"] = ag_to_HDurban["geometry"].astype('str')
ag_to_HDurban["geometry"]  = ag_to_HDurban["geometry"] .str.strip('POINT ')
ag_to_HDurban["geometry"]  = ag_to_HDurban["geometry"] .str.strip('()')
XY = ag_to_HDurban["geometry"] .str.split(" ", expand=True)
ag_to_HDurban["X"] = XY[0]
ag_to_HDurban["Y"] = XY[1]
ag_to_HDurban["X"] = ag_to_HDurban["X"].astype('float')
ag_to_HDurban["Y"] = ag_to_HDurban["Y"].astype('float')
ag_to_HDurban = ag_to_HDurban.drop(columns=['geometry'])
ag_to_HDurban.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6852701 entries, 0 to 6852700
Data columns (total 11 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0_x  int64  
 1   ID            int64  
 2   LC_90m        int64  
 3   NN            int64  
 4   NN_dist       float64
 5   Unnamed: 0_y  int64  
 6   ag_HD1F10     float64
 7   ag_HD2F10     float64
 8   ag_HD3F10     float64
 9   X             float64
 10  Y             float64
dtypes: float64(6), int64(5)
memory usage: 627.4 MB


In [16]:
ag_to_HDurban = ag_to_HDurban.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'])
ag_to_HDurban.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6852701 entries, 0 to 6852700
Data columns (total 9 columns):
 #   Column     Dtype  
---  ------     -----  
 0   ID         int64  
 1   LC_90m     int64  
 2   NN         int64  
 3   NN_dist    float64
 4   ag_HD1F10  float64
 5   ag_HD2F10  float64
 6   ag_HD3F10  float64
 7   X          float64
 8   Y          float64
dtypes: float64(6), int64(3)
memory usage: 522.8 MB


In [64]:
geometry = [Point(xy) for xy in zip(hamlet_to_HDurban.X, hamlet_to_HDurban.Y)]
crs = "EPSG:4326"
hamlet_to_HDurban = GeoDataFrame(hamlet_to_HDurban, crs=crs, geometry=geometry) 
hamlet_to_HDurban.to_file(driver='ESRI Shapefile', filename='SEN-Cdrive/outputs/hamlet_to_HDurban_flood10.shp') 

### Combine with cost-distance raster travel times from origins to road node.

In [9]:
ag_to_HDurban = os.path.join(out_pth, "ag_to_HDurban.csv")
ag_to_HDurban = pd.read_csv(ag_to_HDurban)

In [10]:
LC_value = gpd.read_file("R:/SEN/GEO/Team/Projects/Sen_TransportOV/LC_value/LC_value.shp")
LC_value.head(50)

,ID,LC_90m,NSnomax,geometry
0,6851351,1700,96.0536,POINT (308243.986 1391596.205)
1,6851352,1700,33.9541,POINT (308596.067 1391593.845)
2,6851353,1700,32.1907,POINT (308684.087 1391593.256)
3,6851354,1700,31.1915,POINT (308771.020 1391592.675)
4,6851355,1700,29.5983,POINT (308859.041 1391592.086)
5,6851356,1700,28.7644,POINT (308947.061 1391591.498)
6,6851357,1700,28.0385,POINT (309035.081 1391590.909)
7,6851358,1700,26.1250,POINT (309123.101 1391590.322)
8,6851359,1700,24.1504,POINT (309211.121 1391589.734)
9,6851360,1700,23.3528,POINT (309299.140 1391589.147)


In [11]:
LC_value = pd.DataFrame(LC_value)
LC_value.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6852701 entries, 0 to 6852700
Data columns (total 4 columns):
 #   Column    Dtype   
---  ------    -----   
 0   ID        int64   
 1   LC_90m    int64   
 2   NSnomax   float64 
 3   geometry  geometry
dtypes: float64(1), geometry(1), int64(2)
memory usage: 209.1 MB


In [13]:
# Checking to make sure the ID fields are the same since the dataframes are sorted by different values.
print(LC_value['ID'].min())
print(LC_value['ID'].max())
print(ag_to_HDurban['ID'].min())
print(ag_to_HDurban['ID'].max())

17
7090381
17
7090381


In [12]:
ag_to_HDurban.head()

,Unnamed: 0,ID,LC_90m,geometry,NN,NN_dist,ag_HD1,ag_HD2,ag_HD3
0,0,17,1700,POINT (-15.549972323895453 16.693437843999725),3656617769,6542.678694,163.910812,164.865095,170.548642
1,1,18,1700,POINT (-15.54916384014042 16.693437844000037),3656617769,6511.929511,163.910812,164.865095,170.548642
2,2,19,1700,POINT (-15.54835535638528 16.693437844000353),3656617769,6482.181173,163.910812,164.865095,170.548642
3,3,20,1700,POINT (-15.547546872630239 16.69343784400067),3656617769,6453.447521,163.910812,164.865095,170.548642
4,4,21,1700,POINT (-15.5467383888751 16.693437844000975),3656617804,6425.455616,163.935033,164.889316,170.572863


In [14]:
LC_value = pd.merge(LC_value, ag_to_HDurban, on='ID',how='left')
LC_value

,ID,LC_90m_x,NSnomax,geometry_x,Unnamed: 0,LC_90m_y,geometry_y,NN,NN_dist,ag_HD1,ag_HD2,ag_HD3
0,6851351,1700,96.0536,POINT (308243.986 1391596.205),6617156,1700,POINT (-16.763510008381953 12.583110002062043),3789990598,12915.661914,69.855921,73.224230,206.890106
1,6851352,1700,33.9541,POINT (308596.067 1391593.845),6617157,1700,POINT (-16.760270008384573 12.583110002062732),2289244579,12885.791642,51.759525,55.127833,188.793710
2,6851353,1700,32.1907,POINT (308684.087 1391593.256),6617158,1700,POINT (-16.759460008385226 12.583110002062906),2289244579,12816.976181,51.759525,55.127833,188.793710
3,6851354,1700,31.1915,POINT (308771.020 1391592.675),6617159,1700,POINT (-16.75866000838587 12.583110002063075),2289244579,12749.242409,51.759525,55.127833,188.793710
4,6851355,1700,29.5983,POINT (308859.041 1391592.086),6617160,1700,POINT (-16.75785000838653 12.583110002063249),2289244579,12680.900778,51.759525,55.127833,188.793710
...,...,...,...,...,...,...,...,...,...,...,...,...
6852696,145748,1700,-9999.0000,POINT (871945.071 1828391.715),137502,1700,POINT (-11.514830011852355 16.509100005456542),9152377947,174041.569716,466.944431,467.898715,473.582262
6852697,145749,1700,-9999.0000,POINT (872118.223 1828394.712),137503,1700,POINT (-11.513210011853774 16.50910000545723),9152377947,174179.132045,466.944431,467.898715,473.582262
6852698,147408,1700,-9999.0000,POINT (872553.128 1828223.816),139141,1700,POINT (-11.509170011857845 16.507490005458262),9152377947,174414.410543,466.944431,467.898715,473.582262
6852699,151730,1700,-9999.0000,POINT (872819.573 1827779.585),143370,1700,POINT (-11.506750011861318 16.50344000545756),9152377947,174349.402549,466.944431,467.898715,473.582262


In [ ]:
# The road network includes nodes outside of the country, and the cost-distance raster does not. So we can expect some NaN values.
# Make sure they show up as NaN when creating multi-modal travel times.
LC_value['NSnomax'].replace({-9999: np.nan},inplace =True)

In [17]:
LC_value['ag_HD1mm'] = 0 # mm for multi-modal
LC_value['ag_HD1mm'] = LC_value['ag_HD1'] + LC_value['NSnomax']
# File size is huge. Remove unnecessary data. These data are still saved in the ag_to_HDurban files.
LC_value = LC_value.drop(columns=['LC_90m_x', 'LC_90m_y', 'ag_HD2', 'ag_HD3', 'NN', 'NN_dist']) 
LC_value.rename(columns={'geometry_x': 'geom_UTM', 'geometry_y':'geom_WGS'}, inplace=True)
LC_value

,ID,NSnomax,geom_UTM,Unnamed: 0,geom_WGS,ag_HD1,ag_HD1mm
0,6851351,96.0536,POINT (308243.986 1391596.205),6617156,POINT (-16.763510008381953 12.583110002062043),69.855921,165.909521
1,6851352,33.9541,POINT (308596.067 1391593.845),6617157,POINT (-16.760270008384573 12.583110002062732),51.759525,85.713625
2,6851353,32.1907,POINT (308684.087 1391593.256),6617158,POINT (-16.759460008385226 12.583110002062906),51.759525,83.950225
3,6851354,31.1915,POINT (308771.020 1391592.675),6617159,POINT (-16.75866000838587 12.583110002063075),51.759525,82.951025
4,6851355,29.5983,POINT (308859.041 1391592.086),6617160,POINT (-16.75785000838653 12.583110002063249),51.759525,81.357825
...,...,...,...,...,...,...,...
6852696,145748,NaN,POINT (871945.071 1828391.715),137502,POINT (-11.514830011852355 16.509100005456542),466.944431,NaN
6852697,145749,NaN,POINT (872118.223 1828394.712),137503,POINT (-11.513210011853774 16.50910000545723),466.944431,NaN
6852698,147408,NaN,POINT (872553.128 1828223.816),139141,POINT (-11.509170011857845 16.507490005458262),466.944431,NaN
6852699,151730,NaN,POINT (872819.573 1827779.585),143370,POINT (-11.506750011861318 16.50344000545756),466.944431,NaN


In [18]:
LC_value = LC_value.drop(columns=['Unnamed: 0']) # Forgot that one in previous code block.
crs = "EPSG:31028"
LC_value = GeoDataFrame(LC_value, crs=crs, geometry='geom_UTM') 
LC_value.to_file(driver='ESRI Shapefile', filename='R:/SEN/GEO/Team/Projects/Sen_TransportOV/LC_value/LC_value_mm.shp') 

In [22]:
# Add flood scenario (1 in 10)
ag_to_HDurban = gpd.read_file("C:/Users/wb527163/GEO-Cdrive-Grace/SEN-Cdrive/outputs/ag_to_HDurban_flood10.shp")
ag_to_HDurban.head(50)

,ID,LC_90m,NN,NN_dist,ag_HD1F10,ag_HD2F10,ag_HD3F10,X,Y,geometry
0,17,1700,3656617769,6542.678694,35796.110245,35797.064528,35802.748075,-15.549972,16.693438,POINT (-15.54997 16.69344)
1,18,1700,3656617769,6511.929511,35796.110245,35797.064528,35802.748075,-15.549164,16.693438,POINT (-15.54916 16.69344)
2,19,1700,3656617769,6482.181173,35796.110245,35797.064528,35802.748075,-15.548355,16.693438,POINT (-15.54836 16.69344)
3,20,1700,3656617769,6453.447521,35796.110245,35797.064528,35802.748075,-15.547547,16.693438,POINT (-15.54755 16.69344)
4,21,1700,3656617804,6425.455616,35796.134466,35797.088749,35802.772296,-15.546738,16.693438,POINT (-15.54674 16.69344)
5,22,1700,3656617804,6398.482402,35796.134466,35797.088749,35802.772296,-15.545930,16.693438,POINT (-15.54593 16.69344)
6,23,1700,3656617804,6372.561458,35796.134466,35797.088749,35802.772296,-15.545121,16.693438,POINT (-15.54512 16.69344)
7,24,1700,3656617804,6347.705675,35796.134466,35797.088749,35802.772296,-15.544313,16.693438,POINT (-15.54431 16.69344)
8,25,1700,3656617804,6323.927611,35796.134466,35797.088749,35802.772296,-15.543504,16.693438,POINT (-15.54350 16.69344)
9,26,1700,3656617804,6301.239467,35796.134466,35797.088749,35802.772296,-15.542696,16.693438,POINT (-15.54270 16.69344)


In [23]:
print(LC_value['ID'].min())
print(LC_value['ID'].max())
print(ag_to_HDurban['ID'].min())
print(ag_to_HDurban['ID'].max())

17
7090381
17
7090381


In [ ]:
ag_to_HDurban = pd.DataFrame(ag_to_HDurban)

In [29]:
LC_value = pd.merge(LC_value, ag_to_HDurban, on='ID',how='left')
LC_value['ag_HD1F10mm'] = 0 # mm for multi-modal
LC_value['ag_HD1F10mm'] = LC_value['ag_HD1F10'] + LC_value['NSnomax']
LC_value

,ID,NSnomax,geom_UTM,ag_HD1,ag_HD1mm,LC_90m,NN,NN_dist,ag_HD1F10,ag_HD2F10,ag_HD3F10,X,Y,geometry,ag_HD1F10mm
0,6851351,96.0536,POINT (308243.986 1391596.205),69.855921,165.909521,1700,3789990598,12915.661914,70.207052,73.575361,253.114965,-16.76351,12.58311,POINT (-16.76351 12.58311),166.260652
1,6851352,33.9541,POINT (308596.067 1391593.845),51.759525,85.713625,1700,2289244579,12885.791642,52.110656,55.478964,235.018569,-16.76027,12.58311,POINT (-16.76027 12.58311),86.064756
2,6851353,32.1907,POINT (308684.087 1391593.256),51.759525,83.950225,1700,2289244579,12816.976181,52.110656,55.478964,235.018569,-16.75946,12.58311,POINT (-16.75946 12.58311),84.301356
3,6851354,31.1915,POINT (308771.020 1391592.675),51.759525,82.951025,1700,2289244579,12749.242409,52.110656,55.478964,235.018569,-16.75866,12.58311,POINT (-16.75866 12.58311),83.302156
4,6851355,29.5983,POINT (308859.041 1391592.086),51.759525,81.357825,1700,2289244579,12680.900778,52.110656,55.478964,235.018569,-16.75785,12.58311,POINT (-16.75785 12.58311),81.708956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6852696,145748,NaN,POINT (871945.071 1828391.715),466.944431,NaN,1700,9152377947,174041.569716,142251.364944,142252.319228,142258.002775,-11.51483,16.50910,POINT (-11.51483 16.50910),NaN
6852697,145749,NaN,POINT (872118.223 1828394.712),466.944431,NaN,1700,9152377947,174179.132045,142251.364944,142252.319228,142258.002775,-11.51321,16.50910,POINT (-11.51321 16.50910),NaN
6852698,147408,NaN,POINT (872553.128 1828223.816),466.944431,NaN,1700,9152377947,174414.410543,142251.364944,142252.319228,142258.002775,-11.50917,16.50749,POINT (-11.50917 16.50749),NaN
6852699,151730,NaN,POINT (872819.573 1827779.585),466.944431,NaN,1700,9152377947,174349.402549,142251.364944,142252.319228,142258.002775,-11.50675,16.50344,POINT (-11.50675 16.50344),NaN


In [30]:
LC_value = LC_value.drop(columns=['LC_90m', 'ag_HD2F10', 'ag_HD3F10', 'NN', 'NN_dist', 'geometry'])
LC_value

,ID,NSnomax,geom_UTM,ag_HD1,ag_HD1mm,ag_HD1F10,X,Y,ag_HD1F10mm
0,6851351,96.0536,POINT (308243.986 1391596.205),69.855921,165.909521,70.207052,-16.76351,12.58311,166.260652
1,6851352,33.9541,POINT (308596.067 1391593.845),51.759525,85.713625,52.110656,-16.76027,12.58311,86.064756
2,6851353,32.1907,POINT (308684.087 1391593.256),51.759525,83.950225,52.110656,-16.75946,12.58311,84.301356
3,6851354,31.1915,POINT (308771.020 1391592.675),51.759525,82.951025,52.110656,-16.75866,12.58311,83.302156
4,6851355,29.5983,POINT (308859.041 1391592.086),51.759525,81.357825,52.110656,-16.75785,12.58311,81.708956
...,...,...,...,...,...,...,...,...,...
6852696,145748,NaN,POINT (871945.071 1828391.715),466.944431,NaN,142251.364944,-11.51483,16.50910,NaN
6852697,145749,NaN,POINT (872118.223 1828394.712),466.944431,NaN,142251.364944,-11.51321,16.50910,NaN
6852698,147408,NaN,POINT (872553.128 1828223.816),466.944431,NaN,142251.364944,-11.50917,16.50749,NaN
6852699,151730,NaN,POINT (872819.573 1827779.585),466.944431,NaN,142251.364944,-11.50675,16.50344,NaN


In [32]:
# Shapefiles don't allow columns longer than 10 characters. The ag_ prefix is redundant on the ag shapefile.
LC_value.rename(columns={'ag_HD1': 'HD1', 'ag_HD1mm':'HD1mm', 'ag_HD1F10':'HD1F10', 'ag_HD1F10mm':'HD1F10mm'}, inplace=True)
LC_value.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 6852701 entries, 0 to 6852700
Data columns (total 9 columns):
 #   Column    Dtype   
---  ------    -----   
 0   ID        int64   
 1   NSnomax   float64 
 2   geom_UTM  geometry
 3   HD1       float64 
 4   HD1mm     float64 
 5   HD1F10    float64 
 6   X         float64 
 7   Y         float64 
 8   HD1F10mm  float64 
dtypes: float64(7), geometry(1), int64(1)
memory usage: 522.8 MB


In [33]:
LC_value['HD1dif'] = 0 # dif for difference between the two travel times (in minutes)
LC_value['HD1pc'] = 0 # pc for percent change
LC_value['HD1dif'] = LC_value['HD1F10mm'] - LC_value['HD1mm'] # Travel time is X minutes longer in flood conditions.
LC_value['HD1pc'] = LC_value['HD1dif'] / LC_value['HD1mm'] * 100 # Travel time is X percent longer in flood conditions.
LC_value.head()

,ID,NSnomax,geom_UTM,HD1,HD1mm,HD1F10,X,Y,HD1F10mm,HD1dif,HD1pc
0,6851351,96.0536,POINT (308243.986 1391596.205),69.855921,165.909521,70.207052,-16.76351,12.58311,166.260652,0.351131,0.211640
1,6851352,33.9541,POINT (308596.067 1391593.845),51.759525,85.713625,52.110656,-16.76027,12.58311,86.064756,0.351131,0.409656
2,6851353,32.1907,POINT (308684.087 1391593.256),51.759525,83.950225,52.110656,-16.75946,12.58311,84.301356,0.351131,0.418261
3,6851354,31.1915,POINT (308771.020 1391592.675),51.759525,82.951025,52.110656,-16.75866,12.58311,83.302156,0.351131,0.423299
4,6851355,29.5983,POINT (308859.041 1391592.086),51.759525,81.357825,52.110656,-16.75785,12.58311,81.708956,0.351131,0.431588


In [34]:
crs = "EPSG:31028"
LC_value = GeoDataFrame(LC_value, crs=crs, geometry='geom_UTM') 
LC_value.to_file(driver='ESRI Shapefile', filename='R:/SEN/GEO/Team/Projects/Sen_TransportOV/LC_value/LC_value_mmF10.shp') 

### Analyze results using population and poverty indicators.

In [35]:
region = gpd.read_file("R:\SEN\GEO\Boundaries\SEN_AdminBoundaries_candidate.gdb", layer = 'sen_admbnda_adm1_1m_gov_ocha_20190426')
departement = gpd.read_file("R:\SEN\GEO\Boundaries\SEN_AdminBoundaries_candidate.gdb", layer = 'sen_admbnda_adm2_1m_gov_ocha_20190426')
print(region.head(), end="\n")
print(departement.head(), end="\n")

  admin1Name_fr admin1Pcode admin1RefName admin1AltName1_fr admin1AltName2_fr  \
0         Dakar        SN01          None              None              None   
1      Diourbel        SN02          None              None              None   
2        Fatick        SN03          None              None              None   
3      Kaffrine        SN04          None              None              None   
4       Kaolack        SN05          None              None              None   

  admin0Name_fr admin0Pcode                       date  \
0       Senegal          SN  2017-08-04T00:00:00+00:00   
1       Senegal          SN  2017-08-04T00:00:00+00:00   
2       Senegal          SN  2017-08-04T00:00:00+00:00   
3       Senegal          SN  2017-08-04T00:00:00+00:00   
4       Senegal          SN  2017-08-04T00:00:00+00:00   

                     validOn validTo  Shape_Length  Shape_Area  \
0  2019-04-26T00:00:00+00:00    None      1.494337    0.045667   
1  2019-04-26T00:00:00+00:00    

In [37]:
departement = departement.to_crs("EPSG:31028")
LC_admin = gpd.sjoin_nearest(LC_value, departement, how="left", max_distance=1) 
LC_admin.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 6852701 entries, 0 to 6852700
Data columns (total 26 columns):
 #   Column             Dtype   
---  ------             -----   
 0   ID                 int64   
 1   NSnomax            float64 
 2   geom_UTM           geometry
 3   HD1                float64 
 4   HD1mm              float64 
 5   HD1F10             float64 
 6   X                  float64 
 7   Y                  float64 
 8   HD1F10mm           float64 
 9   HD1dif             float64 
 10  HD1pc              float64 
 11  index_right        float64 
 12  admin2Name_fr      object  
 13  admin2Pcode        object  
 14  admin2RefName      object  
 15  admin2AltName1_fr  object  
 16  admin2AltName2_fr  object  
 17  admin1Name_fr      object  
 18  admin1Pcode        object  
 19  admin0Name_fr      object  
 20  admin0Pcode        object  
 21  date               object  
 22  validOn            object  
 23  validTo            object  
 24  Shape_Length    

In [40]:
G = nx.read_gpickle("SEN-Cdrive/G_clean_processed.pickle")

In [41]:
nx.write_shp(G, 'quick_roadgraph.shp')

C:\Users\wb527163\AppData\Local\Temp/ipykernel_24448/2046061119.py:1: DeprecationWarning: write_shp is deprecated and will be removed in 3.0.See https://networkx.org/documentation/latest/auto_examples/index.html#geospatial.
  nx.write_shp(G, 'quick_roadgraph.shp')


TypeError: 'int' object is not subscriptable